# Training an XOR network using PySyft

In this notebook we'll be training an XOR network using PySyft. 
What's special about this is that the training  is DP- Protected


In [13]:
%load_ext autoreload
%autoreload 2
import syft as sy
import numpy as np
sy.logger.remove()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Logging into the domain Nodes

In [14]:
fb = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)

Connecting to localhost... done! 	 Logging into bold_hotz... done!


In [15]:
fb.datasets

Idx,Name,Description,Assets,Id
[0],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",453b0424-7155-4a0e-9fe8-4bc85511398e


In [16]:
fb.privacy_budget

100.0

In [17]:
X = fb.datasets[-1]["training_data"]
y = fb.datasets[-1]["training_targets"]

In [18]:
X

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

 (The data printed above is synthetic - it's an imitation of the real data.)

In [19]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [20]:
layer0_weights = 2*np.random.random((3,4)) - 1
layer1_weights = 2*np.random.random((4,1)) - 1

In [21]:
for j in range(2):
    # Forward propagation
    layer1_inputs = relu(X @ layer0_weights)
    layer2_inputs = relu(layer1_inputs @ layer1_weights) 
    
    # Calculate errors
    layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True)
    layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True)
    
    # Update weights
    layer1_weights  = layer1_inputs.T @ layer2_inputs_delta + layer1_weights
    layer0_weights =  X.T @ layer1_inputs_delta  + layer0_weights

In [26]:
layer0_weights = layer0_weights.publish(sigma=2e5)
layer1_weights = layer1_weights.publish(sigma=2e5)

In [29]:
# Layer 0 weights
print(layer0_weights)

[[[ 2.9335704   2.87194002  0.14283084 -2.44582303]
  [ 0.45796776 -6.48413099 -2.06327181 -4.29448732]
  [-5.91165136 -4.62117212  0.40206858  2.6372999 ]]]


In [30]:
# Layer 1 weights
print(layer1_weights)

[[[-4.76169182]
  [-4.31603407]
  [-3.3849177 ]
  [-2.72175184]]]


<hr>

Let's see how our privacy budget changed as a result of training for a single epoch:

In [25]:
fb.privacy_budget

84.46756674937654

And voila! We've trained a neural network using PySyft's adversaril differential privacy system and its secure multiparty computation system working in tandem.


<hr>
